In [ ]:
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
import numpy as np
import os
import joblib
import warnings
from tqdm import tqdm

# Suppress ignorable warnings
warnings.filterwarnings('ignore')

def load_data(file_path='../Data/3mon.csv'):
 
    print(f"Loading data from {file_path}...")
    try:
        df = pd.read_csv(file_path, parse_dates=['created_datetime'])
        if df.empty:
            raise ValueError("Loaded dataset is empty")

        print(f"Initial data shape: {df.shape}")
        
        df = df.rename(columns={
            'id': 'Id', 'created_datetime': 'local_time_col', 'water_meter_code': 'meter_code',
            'flow_rate': 'fr', 'flow_volume': 'fv', 'flow_today': 'today', 'net_total': 'net_total'
        })
        
        df = df.sort_values(['meter_code', 'local_time_col'])
        df['fv'] = df.groupby('meter_code')['fv'].transform(lambda x: x.ffill())
        df[['fr', 'fv', 'net_total', 'today']] = df[['fr', 'fv', 'net_total', 'today']].fillna(0)
        
        print(f"Date range: {df['local_time_col'].min()} to {df['local_time_col'].max()}")
        print(f"Number of unique meters: {df['meter_code'].nunique()}")

        df = df.dropna(subset=['local_time_col', 'meter_code', 'fv'])
        print("Data loaded and preprocessed successfully.")
        return df
        
    except Exception as e:
        print(f"Error loading data: {str(e)}")
        raise

def create_features(df, target_col='fv'):

    df = df.copy()
    df['hour'] = df['local_time_col'].dt.hour
    df['day_of_week'] = df['local_time_col'].dt.dayofweek
    df['day_of_month'] = df['local_time_col'].dt.day
    df['month'] = df['local_time_col'].dt.month
    df['quarter'] = df['local_time_col'].dt.quarter
    df['year'] = df['local_time_col'].dt.year
    df['is_weekend'] = (df['local_time_col'].dt.dayofweek >= 5).astype(int)
    
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    df['day_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
    df['day_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    
    return df

def create_lag_rolling_features(df, target_col='fv'):

    df = df.copy()
    lags = [1, 2, 3, 6, 12, 24, 32]
    for lag in lags:
        df[f'{target_col}_lag_{lag}'] = df[target_col].shift(lag)
        
    windows = [3, 6, 12, 24]
    for window in windows:
        df[f'{target_col}_rolling_mean_{window}'] = df[target_col].rolling(window=window).mean()
        df[f'{target_col}_rolling_std_{window}'] = df[target_col].rolling(window=window).std()
        df[f'{target_col}_rolling_min_{window}'] = df[target_col].rolling(window=window).min()
        df[f'{target_col}_rolling_max_{window}'] = df[target_col].rolling(window=window).max()
        
    return df

def xgboost_forecast(df_original, meter_code, granularity, forecast_period=10, target_col='fv'):
  
    try:
        freq = 'D' if granularity == "Daily" else 'H'
        MIN_SAMPLES_FOR_TUNING = 20 if granularity == "Daily" else 240

        # Data Preparation
        df_resampled = df_original.set_index('local_time_col').resample(freq)[target_col].mean().fillna(0).reset_index()

        if len(df_resampled) < MIN_SAMPLES_FOR_TUNING:
            raise ValueError(f"Insufficient data for meter {meter_code}.")

        df_with_features = create_features(df_resampled, target_col=target_col)
        df_with_features = create_lag_rolling_features(df_with_features, target_col=target_col)
        
        # We keep rows with NaNs and let XGBoost handle them
        
        min_test_size = max(forecast_period, int(len(df_with_features) * 0.2))
        split_point = len(df_with_features) - min_test_size
        train_df = df_with_features.iloc[:split_point]
        test_df = df_with_features.iloc[split_point:]

        feature_cols = [col for col in train_df.columns if col not in ['local_time_col', target_col]]
        X_train, y_train = train_df[feature_cols], train_df[target_col]
        X_test, y_test = test_df[feature_cols], test_df[target_col]

        # Check data size before tuning
        # MIN_SAMPLES_FOR_TUNING = 20
        if len(train_df) < MIN_SAMPLES_FOR_TUNING:
            tqdm.write(f"WARNING: Training data for {meter_code} ({granularity}) is too small ({len(train_df)} samples) for tuning. Using default parameters.")
            model_xgb = xgb.XGBRegressor(
                n_estimators=500, max_depth=5, learning_rate=0.05, subsample=0.8,
                colsample_bytree=0.8, random_state=42, n_jobs=-1, early_stopping_rounds=10
            )
            model_xgb.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)
        else:
            # --- UPDATED PARAMETER GRID WITH REGULARIZATION ---
            param_grid = {
                'n_estimators': [100, 500, 1000],
                'max_depth': [3, 5, 7],
                'learning_rate': [0.01, 0.05, 0.1],
                'subsample': [0.7, 0.8, 0.9],
                'colsample_bytree': [0.7, 0.8, 0.9],
                'gamma': [0, 0.1, 0.5],             # Added for regularization
                'reg_alpha': [0, 0.1, 0.5],          # Added for L1 regularization
                'reg_lambda': [1, 1.5, 2]            # Added for L2 regularization
            }
            tscv = TimeSeriesSplit(n_splits=5)
            xgb_estimator = xgb.XGBRegressor(random_state=42, n_jobs=-1, early_stopping_rounds=10)
            
            random_search = RandomizedSearchCV(
                estimator=xgb_estimator, param_distributions=param_grid, n_iter=25,
                scoring='neg_root_mean_squared_error', cv=tscv, verbose=0, random_state=42
            )
            
            fit_params = {"eval_set": [(X_test, y_test)], "verbose": False}
            random_search.fit(X_train, y_train, **fit_params)
            
            # tqdm.write(f"\nBest parameters for {meter_code} ({granularity}): {random_search.best_params_}")
            model_xgb = random_search.best_estimator_

        # --- Forecasting ---
        future_dates = pd.date_range(
            start=df_with_features['local_time_col'].max() + pd.Timedelta(1, unit=freq),
            periods=forecast_period, freq=freq
        )
        
        forecast_results = []
        history = df_with_features.copy()

        for future_date in future_dates:
            future_df = pd.DataFrame({'local_time_col': [future_date]})
            temp_df = pd.concat([history, future_df], ignore_index=True)
            temp_df_features = create_features(temp_df, target_col=target_col)
            temp_df_features = create_lag_rolling_features(temp_df_features, target_col=target_col)
            X_future = temp_df_features.iloc[-1:][feature_cols]
            y_pred = model_xgb.predict(X_future)[0]
            forecast_results.append({'ds': future_date, 'yhat': y_pred})
            # A more robust way to append to history
            new_row = {col: np.nan for col in history.columns}
            new_row['local_time_col'] = future_date
            new_row[target_col] = y_pred
            history = pd.concat([history, pd.DataFrame([new_row])], ignore_index=True)


        forecast_df = pd.DataFrame(forecast_results)
        
        train_df['yhat'] = model_xgb.predict(X_train)
        test_df['yhat'] = model_xgb.predict(X_test)
        
        # --- Plot Generation ---
        if granularity == "Daily":
            plt.figure(figsize=(14, 7))
            plt.plot(train_df['local_time_col'], train_df[target_col], 'b-', label='Actual (Train)')
            plt.plot(test_df['local_time_col'], test_df[target_col], 'g-', label='Actual (Test)')
            plt.plot(train_df['local_time_col'], train_df['yhat'], 'r--', alpha=0.8, label='XGBoost Forecast')
            plt.plot(test_df['local_time_col'], test_df['yhat'], 'r--', alpha=0.8)
            plt.plot(forecast_df['ds'], forecast_df['yhat'], 'r--', alpha=0.8)
            plt.axvline(train_df['local_time_col'].max(), color='k', linestyle=':', label='Train/Test Split')
            plt.axvline(test_df['local_time_col'].max(), color='k', linestyle=':', label='Test/Forecast Split')
            plt.xlabel('Date')
            plt.ylabel(f'Flow Rate ({target_col})')
            plt.title(f'XGBoost Forecast — {meter_code} in {granularity}')
            plt.grid(alpha=0.3)
            plt.legend()
        else: # Hourly
            fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 12), gridspec_kw={'height_ratios': [2, 1]})
            ax1.plot(train_df['local_time_col'], train_df[target_col], 'b-', label='Actual (Train)')
            ax1.plot(test_df['local_time_col'], test_df[target_col], 'g-', label='Actual (Test)')
            ax1.plot(train_df['local_time_col'], train_df['yhat'], 'r-', alpha=0.8, label='XGBoost Forecast')
            ax1.plot(test_df['local_time_col'], test_df['yhat'], 'r-', alpha=0.8)
            ax1.plot(forecast_df['ds'], forecast_df['yhat'], 'r-', alpha=0.8)
            ax1.axvline(train_df['local_time_col'].max(), color='k', linestyle='--', label='Train/Test Split')
            ax1.axvline(test_df['local_time_col'].max(), color='k', linestyle='--', label='Test/Forecast Split')
            ax1.set_title(f'XGBoost Forecast — {meter_code} (Full Hourly Data)')
            ax1.set_xlabel('Date')
            ax1.set_ylabel(f'Flow Rate ({target_col})')
            ax1.legend()
            ax1.grid(alpha=0.3)
            
            forecast_start = df_with_features['local_time_col'].max()
            today_start = forecast_start.floor('D')
            today_end = today_start + pd.Timedelta(days=1)
            today_original = df_with_features[(df_with_features['local_time_col'] >= today_start) & (df_with_features['local_time_col'] <= forecast_start)]
            remaining_hours_forecast = forecast_df[(forecast_df['ds'] > forecast_start) & (forecast_df['ds'] <= today_end)].copy()
            std_dev = y_test.std()
            remaining_hours_forecast['yhat_lower'] = remaining_hours_forecast['yhat'] - 1.96 * std_dev
            remaining_hours_forecast['yhat_upper'] = remaining_hours_forecast['yhat'] + 1.96 * std_dev
            ax2.plot(today_original['local_time_col'], today_original[target_col], 'b-o', label='Today Original Data')
            ax2.plot(remaining_hours_forecast['ds'], remaining_hours_forecast['yhat'], 'r-s', label='Remaining Hours Forecast')
            ax2.fill_between(remaining_hours_forecast['ds'], remaining_hours_forecast['yhat_lower'], remaining_hours_forecast['yhat_upper'], color='orange', alpha=0.3, label='95% Confidence Interval')
            ax2.set_title("Today's Detail: Original Data + Remaining Hours Forecast")
            ax2.set_xlabel('Time (Hours)')
            ax2.set_ylabel(f'Flow Rate ({target_col})')
            ax2.grid(alpha=0.3)
            ax2.legend()
            hour_ticks = pd.date_range(start=today_start, end=today_end, freq='H')
            ax2.set_xticks(hour_ticks)
            ax2.set_xticklabels([dt.strftime('%H:%M') for dt in hour_ticks])
            ax2.set_xlim(today_start, today_end)

        output_dir = "../Output/xgboost_plot/Fv"
        os.makedirs(output_dir, exist_ok=True)
        safe_meter_code = meter_code.replace(" ", "_").replace("/", "_")
        filepath = os.path.join(output_dir, f"{safe_meter_code}_{granularity}_xgboost_forecast.png")
        plt.savefig(filepath, dpi=300, bbox_inches='tight')
        
        model_dir = "../Models/xgboost_models/Fv"
        os.makedirs(model_dir, exist_ok=True)
        model_filepath = os.path.join(model_dir, f"{safe_meter_code}_{granularity}_xgboost_model.joblib")
        joblib.dump(model_xgb, model_filepath)
        
        metrics = {
            'train_RMSE': np.sqrt(mean_squared_error(y_train, train_df['yhat'])),
            'train_MAE': mean_absolute_error(y_train, train_df['yhat']),
            'test_RMSE': np.sqrt(mean_squared_error(y_test, test_df['yhat'])),
            'test_MAE': mean_absolute_error(y_test, test_df['yhat'])
        }
        
        return forecast_df, plt, metrics
        
    except Exception as e:
        tqdm.write(f"\n\nError in xgboost_forecast for meter {meter_code} ({granularity}): {str(e)}\n")
        return None, None, None

def run_xgboost_training(df_original, unique_meters, base_path):
    print("\n" + "-"*100)
    print("STARTING XGBOOST MODEL TRAINING")
    print("-"*100)
    results = []
    
    for meter_code in tqdm(unique_meters, desc="Processing Meters"):
        try:
            meter_data = df_original[df_original['meter_code'] == meter_code].copy()
            if meter_data.empty:
                tqdm.write(f"No data found for meter {meter_code}")
                continue

            for granularity in ["Daily", "Hourly"]:
                forecast_result = xgboost_forecast(
                    meter_data, meter_code, granularity, forecast_period=7
                )
                
                if forecast_result and all(item is not None for item in forecast_result):
                    forecast, plot, metrics = forecast_result
                    # tqdm.write(f"Performance Metrics for {meter_code} ({granularity}): {metrics}")
                    
                    safe_code = meter_code.replace(' ', '_').replace('/', '_')

                    results.append({
                        'meter_id': meter_code,
                        'granularity': granularity,
                        'forecast': forecast, 
                        'metrics': metrics,
                        'plot_saved': f"../Output/xgboost_plot/Fv/{safe_code}_{granularity}_xgboost_forecast.png",
                        'model_saved': f"../Models/xgboost_models/Fv/{safe_code}_{granularity}_xgboost_model.joblib",
                        **metrics # Unpacks the metrics dict into this one
                    })
                    plt.close()
                

        except Exception as e:
            tqdm.write(f"FATAL ERROR processing meter {meter_code}: {str(e)}")
            continue
    return results

def main():
    BASE_PATH = 'Models'
    print(f"Base directory for models: '{BASE_PATH}'")
    
    try:
        df_original = load_data()
        unique_meters = df_original['meter_code'].unique()
        print(f"Found {len(unique_meters)} unique meters to process.")
        
        min_data_points = 100
        meter_counts = df_original['meter_code'].value_counts()
        sufficient_data_meters = meter_counts[meter_counts >= min_data_points].index.tolist()
        
        if len(sufficient_data_meters) < len(unique_meters):
            print(f"Filtered out {len(unique_meters) - len(sufficient_data_meters)} meters with insufficient data")
        
        if sufficient_data_meters:
            xgboost_results_list = run_xgboost_training(df_original, sufficient_data_meters, BASE_PATH)
            if xgboost_results_list:
                print(f"\nCompleted training for {len(xgboost_results_list)} meter-granularity combinations")

                # --- FIXED: Convert list of dicts to DataFrame ---
                results_df = pd.DataFrame(xgboost_results_list)
                
                print("\n" + "-"*80 + "\nTRAINING SUMMARY\n" + "-"*80)
                print(results_df)

                # --- FIXED: Save the DataFrame to a CSV file ---
                output_dir = "../Output/forecast_result"
                os.makedirs(output_dir, exist_ok=True)
                csv_path = os.path.join(output_dir, "xgboost_fr_results.csv")
                results_df.to_csv(csv_path, index=False)
                print(f"\nResults saved to {csv_path}")

            else:
                print("\nTraining run completed, but no models were successfully trained.")
            
        else:
            print("No meters with sufficient data for training.")
            
    except Exception as e:
        print(f"Error in main execution: {str(e)}")
        raise

if __name__ == "__main__":
    main()

Base directory for models: 'Models'
Loading data from ../Data/3mon.csv...
Initial data shape: (1088132, 10)
Date range: 2025-04-01 00:00:00 to 2025-06-16 15:00:00
Number of unique meters: 161
Data loaded and preprocessed successfully.
Found 161 unique meters to process.

----------------------------------------------------------------------------------------------------
STARTING XGBOOST MODEL TRAINING
----------------------------------------------------------------------------------------------------


Processing Meters:   0%|          | 0/161 [00:11<?, ?it/s]


Best parameters for BFM 19 (Daily): {'subsample': 0.9, 'reg_lambda': 1.5, 'reg_alpha': 0.5, 'n_estimators': 500, 'max_depth': 5, 'learning_rate': 0.01, 'gamma': 0.1, 'colsample_bytree': 0.7}


Processing Meters:   0%|          | 0/161 [00:12<?, ?it/s]

Performance Metrics for BFM 19 (Daily): {'train_RMSE': np.float64(0.0), 'train_MAE': 0.0, 'test_RMSE': np.float64(0.0), 'test_MAE': 0.0}


...



Error in xgboost_forecast for meter AMR-113 (Daily): Insufficient data for meter AMR-113.



Error in xgboost_forecast for meter AMR-113 (Hourly): Insufficient data for meter AMR-113.



Processing Meters:  99%|█████████▉| 160/161 [1:51:05<00:09,  9.33s/it]



Error in xgboost_forecast for meter AMR-194 (Daily): Insufficient data for meter AMR-194.



Error in xgboost_forecast for meter AMR-194 (Hourly): Insufficient data for meter AMR-194.



Processing Meters: 100%|██████████| 161/161 [1:51:05<00:00, 41.40s/it]




Error in xgboost_forecast for meter AMR-95 (Daily): Insufficient data for meter AMR-95.



Error in xgboost_forecast for meter AMR-95 (Hourly): Insufficient data for meter AMR-95.


Completed training for 315 meter-granularity combinations

--------------------------------------------------------------------------------
TRAINING SUMMARY
--------------------------------------------------------------------------------
    meter_id granularity                                           forecast  \
0     BFM 19       Daily            ds  yhat
0 2025-06-17   0.0
1 2025-0...   
1     BFM 19      Hourly                     ds  yhat
0 2025-06-16 16:00...   
2     AMR-05       Daily            ds  yhat
0 2025-06-17   0.0
1 2025-0...   
3     AMR-05      Hourly                     ds  yhat
0 2025-06-16 16:00...   
4     AMR-08       Daily            ds  yhat
0 2025-06-17   0.0
1 2025-0...   
..       ...         ...                                                ...   
310   AMR-12       Daily 